<a href="https://colab.research.google.com/github/dean-daryl/multi-modal-auth-prediction/blob/main/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face_recognition opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=c5e1992f1c39c26702f47b11cb7b897f3a51c16dd9bed8126de2523682deb52c
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [16]:
from google.colab import files

uploaded = files.upload()

Saving download (1).jpeg to download (1).jpeg
Saving download.jpeg to download.jpeg
Saving images (1).jpeg to images (1) (1).jpeg
Saving images (2).jpeg to images (2) (1).jpeg
Saving images (3).jpeg to images (3) (1).jpeg
Saving images (4).jpeg to images (4) (1).jpeg
Saving images.jpeg to images (5).jpeg
Saving inmate 1.jpeg to inmate 1 (2).jpeg
Saving inmate 2.jpeg to inmate 2 (3).jpeg
Saving inmate 3.jpeg to inmate 3 (3).jpeg
Saving inmate 4.jpeg to inmate 4 (3).jpeg
Saving inmate 5.jpeg to inmate 5 (3).jpeg


In [17]:
import cv2
import numpy as np
import os

def extract_face(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (100, 100))
        return face_resized
    return None

In [18]:
faces = []
labels = []

for file in uploaded.keys():
    label = file.split('_')[0]  # e.g., user1
    face = extract_face(file)
    if face is not None:
        faces.append(face.flatten())  # Flatten 2D face into 1D vector
        labels.append(label)
    else:
        print(f"No face found in {file}")

faces = np.array(faces)
labels = np.array(labels)

print("Dataset shape:", faces.shape)

No face found in images (3) (1).jpeg
Dataset shape: (11, 10000)


In [22]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

# Step 1: Reduce dimensions with PCA
# Automatically set components to something safe
n_components = min(10, faces.shape[0], faces.shape[1])
pca = PCA(n_components=n_components)
faces_pca = pca.fit_transform(faces)

# Step 2: Train classifier on the full dataset
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(faces_pca, labels)

print(f"✅ Model trained on {len(labels)} samples with {n_components} PCA components.")


✅ Model trained on 11 samples with 10 PCA components.


In [24]:
def predict_face(image_path):
    face = extract_face(image_path)
    if face is None:
        return "❌ No face found"
    face_pca = pca.transform([face.flatten()])
    pred = model.predict(face_pca)[0]
    confidence = model.predict_proba(face_pca).max()
    return f"✅ Predicted: {pred} with confidence {confidence:.2f}"

# Example usage:
print(predict_face('inmate 1.jpeg'))  # Replace with your test file


✅ Predicted: inmate 1 (2).jpeg with confidence 0.55


In [25]:
import joblib

# Save model and PCA
joblib.dump(model, "face_model.pkl")
joblib.dump(pca, "pca_model.pkl")


['pca_model.pkl']

In [26]:
from google.colab import files

# Download both files
files.download("face_model.pkl")
files.download("pca_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>